In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.metrics import accuracy_score
from resnet50_mlp_fusion import SupCEResNet_Fusion  
from train import train_model
from data_preprocessing import get_data_loaders
import torch.nn as nn
import torch.optim as optim

# Paths
DATA_PATH = '/storage/ice1/shared/d-pace_community/makerspace-datasets/MEDICAL/OLIVES/OLIVES/'
LABEL_DATA_PATH = os.path.join(DATA_PATH, 'Biomarker_Clinical_Data_Images_Updated.csv')

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Define model
model = SupCEResNet_Fusion(
    name='resnet50',
    num_classes=1,  # Regression or binary classification
    num_features=14,  # Number of biomarkers
    in_channels=3  # RGB images
).to(DEVICE)

# Define the criterion and optimizer
criterion = nn.MSELoss()  # Use MSELoss for regression tasks
optimizer = optim.Adam(model.parameters(), lr=1e-4)
train_loader, val_loader = get_data_loaders()
# Train the model
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=DEVICE,
    epochs=10  # Specify the number of epochs
)


/home/hice1/cphan36/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 6/10 - Train Loss: 44.6945, Val Loss: 47.2235
Epoch 7/10 - Train Loss: 41.0312, Val Loss: 44.2377
Epoch 8/10 - Train Loss: 36.5389, Val Loss: 37.5902
Epoch 9/10 - Train Loss: 34.0994, Val Loss: 37.5623
Epoch 10/10 - Train Loss: 31.5153, Val Loss: 35.2049
